# ML Reasearch 
Автор: Киселева Анастсия Денисовна, R4197

In [1]:
# Загрузка библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import mlflow

In [2]:
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Обучение моделей
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from imblearn.pipeline import Pipeline, make_pipeline

In [5]:
# import lightgbm as lgb
from lightgbm import LGBMClassifier

In [6]:
# Указываем пользователя, который делает эксперименты
os.environ['USER'] = 'Anastasia Kiseleva'

In [7]:
# Инициализация MLflow
mlflow.set_tracking_uri("http://178.72.151.49:5050")

In [8]:
mlflow.set_experiment("Insurance Cross Selling Classifier")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1764667633320, experiment_id='2', last_update_time=1764667633320, lifecycle_stage='active', name='Insurance Cross Selling Classifier', tags={}>

# Подготовка данных

### References

<b>Data</b>
- Ссылка на датасет в Kaggle: https://www.kaggle.com/datasets/annantkumarsingh/health-insurance-cross-sell-prediction-data/discussion/516324

### Описание данных
| Название признака | Тип | Описание  |
| :------- | :------ | :------- |
|id |числовой|Уникальный идентификатор клиента|
|Age|числовой|Возраст клиента|
|Gender|бинарный|Пол клиента|
|Driving_License|бинарный|0 для клиентов, не имеющих водительских прав, 1 для клиентов, имеющих водительские права|
|Region_Code|номинальный|Уникальный код региона клиента|
|Previously_Insured|бинарный|0 для клиентов, не имеющих страховки на транспортное средство, 1 для клиентов, имеющих страховку на транспортное средство|
|Vehicle_Age|номинальный|Возраст транспортного средства|
|Vehicle_Damage|бинарный|1: Клиент получил повреждение своего автомобиля в прошлом. 0: Клиент не получал повреждения своего автомобиля в прошлом|
|Annual_Premium|числовой|Сумма, которую клиент должен заплатить в качестве премии в течение года|
|Policy_Sales_Channel|номинальный|Анонимизированный код канала взаимодействия с клиентом, т. е. различные агенты, по почте, по телефону, лично и т. д.|
|Vintage|числовой|Количество дней, в течение которых клиент связан с компанией|
|Response (таргет)|бинарный|1 для заинтересованного клиента, 0 для незаинтересованного клиента|

In [9]:
# Загрузка датасета
df = pd.read_csv('train.csv')

In [10]:
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [11]:
df.shape

(11504798, 12)

In [12]:
# Выделяем обучающий набор 
df = df.head(5000000)
df.shape

(5000000, 12)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 457.8+ MB


In [14]:
# Подсчет пропущенных значений
df.isna().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

В наборе данных не наблюдаются пропущенные значения.

In [15]:
# Сразу отбросим столбец id, т.к. он никакую полезную информацию о клиенте не несет
print('Число уникальных значений признака id:', len(df.id.unique()))
df = df.drop(columns = ['id'])

Число уникальных значений признака id: 5000000


In [16]:
# Описательная статистика признаков в числовом формате
df.describe()

,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06
mean,3.838755e+01,9.980264e-01,2.641528e+01,4.628668e-01,3.046855e+04,1.124040e+02,1.639189e+02,1.230562e-01
std,1.499676e+01,4.438136e-02,1.298993e+01,4.986193e-01,1.647620e+04,5.404819e+01,7.997458e+01,3.285017e-01
min,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.630000e+03,1.000000e+00,1.000000e+01,0.000000e+00
25%,2.400000e+01,1.000000e+00,1.500000e+01,0.000000e+00,2.527900e+04,2.900000e+01,9.900000e+01,0.000000e+00
50%,3.600000e+01,1.000000e+00,2.800000e+01,0.000000e+00,3.182400e+04,1.510000e+02,1.660000e+02,0.000000e+00
75%,4.900000e+01,1.000000e+00,3.500000e+01,1.000000e+00,3.945400e+04,1.520000e+02,2.320000e+02,0.000000e+00
max,8.500000e+01,1.000000e+00,5.200000e+01,1.000000e+00,5.401650e+05,1.630000e+02,2.990000e+02,1.000000e+00


In [17]:
# Описательная статистика категориальных признаков
df.describe(include = ['O'])

,Gender,Vehicle_Age,Vehicle_Damage
count,5000000,5000000,5000000
unique,2,3,2
top,Male,1-2 Year,Yes
freq,2707018,2600661,2514240


In [18]:
# Закодируем оставшиеся категориальные признаки методом Label Encoding
gender = {'Male': 0, 'Female': 1}
df['Gender'] = df['Gender'].map(gender)

vehicle_age = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years':2}
df['Vehicle_Age'] = df['Vehicle_Age'].map(vehicle_age)

vehicle_damage = {'No': 0, 'Yes': 1}
df['Vehicle_Damage'] = df['Vehicle_Damage'].map(vehicle_damage)

In [19]:
# Описательная статистика с учетом закодированных признаков
df.describe()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06
mean,4.585964e-01,3.838755e+01,9.980264e-01,2.641528e+01,4.628668e-01,6.032090e-01,5.028480e-01,3.046855e+04,1.124040e+02,1.639189e+02,1.230562e-01
std,4.982828e-01,1.499676e+01,4.438136e-02,1.298993e+01,4.986193e-01,5.678246e-01,4.999919e-01,1.647620e+04,5.404819e+01,7.997458e+01,3.285017e-01
min,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.630000e+03,1.000000e+00,1.000000e+01,0.000000e+00
25%,0.000000e+00,2.400000e+01,1.000000e+00,1.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.527900e+04,2.900000e+01,9.900000e+01,0.000000e+00
50%,0.000000e+00,3.600000e+01,1.000000e+00,2.800000e+01,0.000000e+00,1.000000e+00,1.000000e+00,3.182400e+04,1.510000e+02,1.660000e+02,0.000000e+00
75%,1.000000e+00,4.900000e+01,1.000000e+00,3.500000e+01,1.000000e+00,1.000000e+00,1.000000e+00,3.945400e+04,1.520000e+02,2.320000e+02,0.000000e+00
max,1.000000e+00,8.500000e+01,1.000000e+00,5.200000e+01,1.000000e+00,2.000000e+00,1.000000e+00,5.401650e+05,1.630000e+02,2.990000e+02,1.000000e+00


## Нормализация данных 

In [20]:
scaler = MinMaxScaler().fit(df)

df_sc = pd.DataFrame(scaler.transform(df), columns = df.columns)
df_sc.describe()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06
mean,4.585964e-01,2.828855e-01,9.980264e-01,5.079862e-01,4.628668e-01,3.016045e-01,5.028480e-01,5.178929e-02,6.876789e-01,5.325915e-01,1.230562e-01
std,4.982828e-01,2.307194e-01,4.438136e-02,2.498064e-01,4.986193e-01,2.839123e-01,4.999919e-01,3.065139e-02,3.336308e-01,2.767287e-01,3.285017e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,6.153846e-02,1.000000e+00,2.884615e-01,0.000000e+00,0.000000e+00,0.000000e+00,4.213493e-02,1.728395e-01,3.079585e-01,0.000000e+00
50%,0.000000e+00,2.461538e-01,1.000000e+00,5.384615e-01,0.000000e+00,5.000000e-01,1.000000e+00,5.431088e-02,9.259259e-01,5.397924e-01,0.000000e+00
75%,1.000000e+00,4.461538e-01,1.000000e+00,6.730769e-01,1.000000e+00,5.000000e-01,1.000000e+00,6.850531e-02,9.320988e-01,7.681661e-01,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


# Создание и обучение моделей

In [21]:
# Разбиение выборки для кросс-валидации
kFold = StratifiedKFold(n_splits = 5, shuffle = False)

In [22]:
X = df_sc[[col for col in df_sc.columns if col != 'Response']]
y = df_sc['Response']

In [23]:
# Выделим категориальные признаки
X['Region_Code_cat'] = X['Region_Code'].astype('category')
X['Policy_Sales_Channel_cat'] = X['Policy_Sales_Channel'].astype('category')
X = X.drop(columns = ['Region_Code', 'Policy_Sales_Channel'])

## LightGBM Classifier

### Baseline Model

In [26]:
# Создаем классификатор с параметрами по умолчанию
lgbm = LGBMClassifier(class_weight = 'balanced')

In [27]:
# Подбираем лучшие параметры дла классификатора, максимизирующие метрику recall
params = {
    'n_estimators': [100, 150, 200],
    'learning_rate': np.arange(0.05, 0.2, step = 0.05),
    'min_data_in_leaf': np.arange(1, 10, 2),
    'subsample' : [0.6, 0.75, 0.8, 1],
    'colsample_bytree': [0.5, 0.75, 1]
         }

random_lgbm = RandomizedSearchCV(lgbm, params, cv = kFold, 
                          scoring = 'roc_auc', n_jobs = 2, n_iter = 5, random_state = 42, error_score = 'raise').fit(X, y)

print('Лучшие гиперпараметры модели:', random_lgbm.best_params_)
print(f'Лучший результат: {random_lgbm.best_score_:.4f}')

[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Info] Number of positive: 492225, number of negative: 3507775
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.510103 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 730
[LightGBM] [Info] Number of data points in the train set: 4000000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] m

In [24]:
# Модель с лучшими параметрами
lgbm = LGBMClassifier (n_estimators = 150, learning_rate = 0.2,
                       min_data_in_leaf = 3,
                       subsample = 0.8,
                       colsample_bytree = 0.5,
                       random_seed = 42,
                       class_weight = 'balanced')

In [25]:
# Оценка других метрик для лучшего классификатора
mlflow.start_run(run_name='Baseline LGBM Gradient Boost')
res = [{'model': 'Baseline LGBM Gradient Boost'}]
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

mlflow.log_param("n_estimators", 150)
mlflow.log_param("learning_rate", 0.2)
mlflow.log_param("subsample", 0.8)
mlflow.log_param("colsample_bytree", 0.5)
mlflow.log_param("min_data_in_leaf", 3)

cv_metrics = cross_validate(lgbm, X, y, scoring = scoring, cv = kFold, n_jobs = -1)
for score in scoring:
    print(f"{score}: {np.mean(cv_metrics['test_' + score]):.4f} ({np.std(cv_metrics['test_' + score]):.4f})")
    res[0][score] = round(np.mean(cv_metrics['test_' + score]), 4)
    mlflow.log_metric(score,  res[0][score])

mlflow.sklearn.log_model(lgbm, "Baseline model") 
mlflow.end_run()

2025/12/13 15:54:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


accuracy: 0.7253 (0.0003)
precision: 0.3004 (0.0003)
recall: 0.9277 (0.0004)
f1: 0.4539 (0.0003)
roc_auc: 0.8782 (0.0003)


2025/12/13 15:54:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Baseline LGBM Gradient Boost at: http://178.72.151.49:5050/#/experiments/2/runs/284d32121b064c40a1f37fd3a6d7cd24
🧪 View experiment at: http://178.72.151.49:5050/#/experiments/2
[LightGBM] [Warning] min_data_in_leaf is set=3, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3
[LightGBM] [Warning] min_data_in_leaf is set=3, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3
[LightGBM] [Info] Number of positive: 492225, number of negative: 3507775
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.805971 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 731
[LightGBM] [Info] Number of data points in the train set: 4000000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] min_data_in_leaf is set=3, min_child_samples=20 will 

In [26]:
from sklearn.metrics import f1_score, make_scorer
f1_scorer = make_scorer(f1_score)

base_lgbm = LGBMClassifier(
    class_weight='balanced',
    random_state=42
)

params_f1 = {
    'n_estimators': [100, 150, 200],
    'learning_rate': np.arange(0.05, 0.2, step=0.05),  
    'min_data_in_leaf': np.arange(1, 10, 2),          
    'subsample': [0.6, 0.75, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.75, 1.0]
}

random_lgbm_f1 = RandomizedSearchCV(
    base_lgbm,
    params_f1,
    cv=kFold,           
    scoring=f1_scorer,  
    n_iter=5,           
    n_jobs=2,
    random_state=42,

    
    error_score='raise'
)


random_lgbm_f1.fit(X, y)

print('Лучшие гиперпараметры (оптимизация по F1):')
print(random_lgbm_f1.best_params_)
print(f'Лучший F1 на кросс-валидации: {random_lgbm_f1.best_score_:.4f}')

best_lgbm_f1 = random_lgbm_f1.best_estimator_

res.append({'model': 'LGBM_optimized_F1'})

mlflow.start_run(run_name='LGBM optimized F1')

for p in ['n_estimators', 'learning_rate', 'subsample',
          'colsample_bytree', 'min_data_in_leaf']:
    mlflow.log_param(p, best_lgbm_f1.get_params()[p])

scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

cv_metrics = cross_validate(
    best_lgbm_f1,
    X, y,
    scoring=scoring,
    cv=kFold,
    n_jobs=2     
)

for score in scoring:
    mean_score = np.mean(cv_metrics['test_' + score])
    std_score = np.std(cv_metrics['test_' + score])
    print(f"{score}: {mean_score:.4f} ({std_score:.4f})")
    
    res[-1][score] = round(mean_score, 4)
    mlflow.log_metric(score, res[-1][score])

mlflow.sklearn.log_model(best_lgbm_f1, "LGBM optimized F1")
mlflow.end_run()


[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Info] Number of positive: 492225, number of negative: 3507775
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.099677 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 731
[LightGBM] [Info] Number of data points in the train set: 4000000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be i

/usr/local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Info] Number of positive: 492225, number of negative: 3507775
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.598106 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 732
[LightGBM] [Info] Number of data points in the train set: 4000000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] m

2025/12/13 17:22:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


accuracy: 0.7237 (0.0004)
precision: 0.2994 (0.0003)
recall: 0.9294 (0.0007)
f1: 0.4529 (0.0003)
roc_auc: 0.8779 (0.0003)


2025/12/13 17:22:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LGBM optimized F1 at: http://178.72.151.49:5050/#/experiments/2/runs/467d8acd73c74abb9555ca48c91ad7d0
🧪 View experiment at: http://178.72.151.49:5050/#/experiments/2


### Результаты обучения моделей

In [27]:
results = pd.DataFrame(res)
results

,model,accuracy,precision,recall,f1,roc_auc
0,Baseline LGBM Gradient Boost,0.7253,0.3004,0.9277,0.4539,0.8782
1,LGBM_optimized_F1,0.7237,0.2994,0.9294,0.4529,0.8779
